# Docker

Ein Jupyter notebook zum Buch "Docker - Software entwickeln und deployen mit Container"

## Installation

Ist auf der Box installiert. Sollte alles laufen.


### Ohne sudo starten

Der login-user ist in der Gruppe __docker__.

In [1]:
id

uid=1000(vagrant) gid=1000(vagrant) Gruppen=1000(vagrant),4(adm),24(cdrom),27(sudo),30(dip),46(plugdev),110(lxd),115(lpadmin),116(sambashare),998(docker)


In [2]:
docker --version

Docker version 1.12.2, build bb80604


In [3]:
sudo service docker restart

## Ein schneller Check

In [4]:
docker version

Client:
 Version:      1.12.2
 API version:  1.24
 Go version:   go1.6.3
 Git commit:   bb80604
 Built:        Tue Oct 11 18:29:41 2016
 OS/Arch:      linux/amd64

Server:
 Version:      1.12.2
 API version:  1.24
 Go version:   go1.6.3
 Git commit:   bb80604
 Built:        Tue Oct 11 18:29:41 2016
 OS/Arch:      linux/amd64


## Erste Schritte

### Images ausführen

In [5]:
docker run debian echo "Hallo Welt"

Hallo Welt


Interaktiver TTY geht unter jupyter nicht. Bitte dies im Terminal ausprobieren.

Im Container dann
```bash
docker run -it debian /bin/bash                                                                                         
root@5772a6081d6e:/# mv /bin /basket                                                                                                      
root@5772a6081d6e:/# ls                                                                                                                   
bash: ls: command not found  
```
ausführen.

In [7]:
docker inspect 5772a6081d6e

[
    {
        "Id": "5772a6081d6e58700d19f5d23b7c7d7229e19f302099502e9fa19221e4db882c",
        "Created": "2016-10-16T19:06:41.150198107Z",
        "Path": "/bin/bash",
        "Args": [],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 7680,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2016-10-16T19:06:41.590560279Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:ddf73f48a05d97e4f473d0b4ccb53383cbb0647d10e34b62d68bfc859cc6bcf9",
        "ResolvConfPath": "/var/lib/docker/containers/5772a6081d6e58700d19f5d23b7c7d7229e19f302099502e9fa19221e4db882c/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/5772a6081d6e58700d19f5d23b7c7d7229e19f302099502e9fa19221e4db882c/hostname",
        "HostsPath": "/var/lib/docker/containers/5772a608

In [8]:
docker inspect 5772a6081d6e | grep IPAddress

            "SecondaryIPAddresses": null,
            "IPAddress": "172.17.0.2",
                    "IPAddress": "172.17.0.2",


In [9]:
docker inspect --format {{.NetworkSettings.IPAddress}} 5772a6081d6e

172.17.0.2


In [11]:
docker diff 5772a6081d6e

C /.wh..wh.plnk
A /.wh..wh.plnk/93.1183573
D /bin
A /basket
A /basket/bash
A /basket/cat
A /basket/chacl
A /basket/chgrp
A /basket/chmod
A /basket/chown
A /basket/cp
A /basket/dash
A /basket/date
A /basket/dd
A /basket/df
A /basket/dir
A /basket/dmesg
A /basket/dnsdomainname
A /basket/domainname
A /basket/echo
A /basket/egrep
A /basket/false
A /basket/fgrep
A /basket/findmnt
A /basket/getfacl
A /basket/grep
A /basket/gunzip
A /basket/gzexe
A /basket/gzip
A /basket/hostname
A /basket/ip
A /basket/journalctl
A /basket/kill
A /basket/ln
A /basket/login
A /basket/loginctl
A /basket/ls
A /basket/lsblk
A /basket/machinectl
A /basket/mkdir
A /basket/mknod
A /basket/mktemp
A /basket/more
A /basket/mount
A /basket/mountpoint
A /basket/mv
A /basket/nisdomainname
A /basket/pidof
A /basket/ping
A /basket/ping6
A /basket/ps
A /basket/pwd
A /basket/rbash
A /basket/readlink
A /basket/rm
A /basket/rmdir
A /basket/run-parts
A /basket/sed
A /basket/setfacl
A /basket/sh
A /basket/sh.distrib
A /basket/sle

In [13]:
docker logs 5772a6081d6e

Stoppen und löschen gleichzeitig.

In [14]:
docker rm -f 5772a6081d6e

5772a6081d6e


In [17]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
da5a70fda7f9        1572110032c9        "/entrypoint.sh"    10 minutes ago      Created                                 gigantic_kirch
48d86be15e38        fd000d9e47bf        "/entrypoint.sh"    15 minutes ago      Created                                 furious_meitner


In [19]:
docker rm -v $(docker ps -aq -f status=exited)

0d7667bf2f9d


In einem Terminal ausführen

```bash
docker run -it --name cowsay --hostname cowsay debian bash

root@cowsay:/# apt-get update 
root@cowsay:/# apt-get install -y cowsay fortune
root@cowsay:/# /usr/games/fortune | /usr/games/cowsay

_________________________________________                 
/ You will be traveling and coming into a\
\ fortune.                               /
-----------------------------------------                 
        \  ^__^      
         \ (oo)\_______      
           (__)\       )\/\
                ||----w |     
                ||     ||
                
root@cowsay:/# exit
```

In [20]:
docker commit cowsay test/cowsayimage

sha256:a24a6ed41f5b189d0f61c40a2ef37fb1d7a20aac1e67ba0468b211fa080df437


In [21]:
docker run test/cowsayimage /usr/games/cowsay "Muh"

 _____
< Muh >
 -----
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


## Images aus Dockerfiles erstellen

In [22]:
cd ~/projects
rm -rf cowsay
mkdir cowsay
cd cowsay
touch Dockerfile
echo -e "FROM debian\n\nRUN apt-get update && apt-get install -y cowsay fortune" > Dockerfile

In [23]:
cat Dockerfile

FROM debian

RUN apt-get update && apt-get install -y cowsay fortune


In [24]:
docker build -t test/cowsay-dockerfile .


Step 1 : FROM debian
 ---> ddf73f48a05d
Step 2 : RUN apt-get update && apt-get install -y cowsay fortune
 ---> Running in a6e3ede363fd
Get:1 http://security.debian.org jessie/updates InRelease [63.1 kB]
Ign http://httpredir.debian.org jessie InRelease
Get:2 http://security.debian.org jessie/updates/main amd64 Packages [391 kB]
Get:3 http://httpredir.debian.org jessie-updates InRelease [142 kB]
Get:4 http://httpredir.debian.org jessie Release.gpg [2373 B]
Get:5 http://httpredir.debian.org jessie Release [148 kB]
Get:6 http://httpredir.debian.org jessie-updates/main amd64 Packages [17.6 kB]
Get:7 http://httpredir.debian.org jessie/main amd64 Packages [9064 kB]
Fetched 9829 kB in 8s (1172 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
The following extra packages will be installed:
  fortunes-min libalgorithm-c3-perl libarchive-extract-perl libcgi-fast-perl
  libcgi-pm-perl libclass-c3-perl libclass-c3-xs-perl libcpan-meta-perl
  libdata-optlist-perl 

In [25]:
docker run test/cowsay-dockerfile /usr/games/cowsay "Muh"

 _____
< Muh >
 -----
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [26]:
echo -e '\nENTRYPOINT ["/usr/games/cowsay"]' >> Dockerfile

In [27]:
cat Dockerfile

FROM debian

RUN apt-get update && apt-get install -y cowsay fortune

ENTRYPOINT ["/usr/games/cowsay"]


In [28]:
docker build -t test/cowsay-dockerfile .


Step 1 : FROM debian
 ---> ddf73f48a05d
Step 2 : RUN apt-get update && apt-get install -y cowsay fortune
 ---> Using cache
 ---> 514e4c9cc808
Step 3 : ENTRYPOINT /usr/games/cowsay
 ---> Running in c8329e0899b8
 ---> 56452449687c
Removing intermediate container c8329e0899b8
Successfully built 56452449687c


In [29]:
docker run test/cowsay-dockerfile "Muh"

 _____
< Muh >
 -----
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [30]:
echo -e '#!/bin/bash\nif [ $# -eq 0 ]; then\n    /usr/games/fortune | /usr/games/cowsay\nelse\n    /usr/games/cowsay "$@"\nfi' > entrypoint.sh
chmod u+x entrypoint.sh
cat entrypoint.sh

#!/bin/bash
if [ $# -eq 0 ]; then
    /usr/games/fortune | /usr/games/cowsay
else
    /usr/games/cowsay "$@"
fi


In [31]:
sed -i 's/^ENTRYPOINT \[\"\/usr\/games\/cowsay\"\]/COPY entrypoint.sh \/\n\nENTRYPOINT \[\"\/entrypoint.sh\"\]/g' Dockerfile
cat Dockerfile

FROM debian

RUN apt-get update && apt-get install -y cowsay fortune

COPY entrypoint.sh /

ENTRYPOINT ["/entrypoint.sh"]


In [32]:
docker build -t test/cowsay-dockerfile .


Step 1 : FROM debian
 ---> ddf73f48a05d
Step 2 : RUN apt-get update && apt-get install -y cowsay fortune
 ---> Using cache
 ---> 514e4c9cc808
Step 3 : COPY entrypoint.sh /
 ---> 6b2436003d28
Removing intermediate container 3addfd462f8b
Step 4 : ENTRYPOINT /entrypoint.sh
 ---> Running in 841efdb64e26
 ---> 4a3a51ff0663
Removing intermediate container 841efdb64e26
Successfully built 4a3a51ff0663


In [33]:
docker run test/cowsay-dockerfile

 _________________________________________
/ People are beginning to notice you. Try \
\ dressing before you leave the house.    /
 -----------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [34]:
docker run test/cowsay-dockerfile "Hallo Muh"

 ___________
< Hallo Muh >
 -----------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


## Mit Registries arbeiten

Mit docker login beim Registry einlogen. Bitte im Terminal eingeben da interaktiv.

In [45]:
echo -e 'FROM debian\n\nMAINTAINER Micheal Schaar <kraeml@sesamestreet.franken.de>\nRUN apt-get update && apt-get install -y cowsay fortune\nCOPY entrypoint.sh /\n\nENTRYPOINT ["/entrypoint.sh"]' > Dockerfile
cat Dockerfile

FROM debian

MAINTAINER Micheal Schaar <kraeml@sesamestreet.franken.de>
RUN apt-get update && apt-get install -y cowsay fortune
COPY entrypoint.sh /

ENTRYPOINT ["/entrypoint.sh"]


In [46]:
docker build -t kraeml/cowsay .


Step 1 : FROM debian
 ---> ddf73f48a05d
Step 2 : MAINTAINER Micheal Schaar <kraeml@sesamestreet.franken.de>
 ---> Using cache
 ---> 64109da3f6b4
Step 3 : RUN apt-get update && apt-get install -y cowsay fortune
 ---> Using cache
 ---> bc0480da28cf
Step 4 : COPY entrypoint.sh /
 ---> ef9be661b2a1
Removing intermediate container 644a22c1df1e
Step 5 : ENTRYPOINT /entrypoint.sh
 ---> Running in dff5d2ae9bf1
 ---> b4a1cd81ef44
Removing intermediate container dff5d2ae9bf1
Successfully built b4a1cd81ef44


In [47]:
docker images

REPOSITORY               TAG                 IMAGE ID            CREATED             SIZE
kraeml/cowsay            latest              b4a1cd81ef44        5 seconds ago       171 MB
test/cowsay-dockerfile   latest              4a3a51ff0663        7 minutes ago       171 MB
<none>                   <none>              56452449687c        7 minutes ago       171 MB
test/cowsayimage         latest              a24a6ed41f5b        8 minutes ago       171 MB
<none>                   <none>              1535d5438a34        20 minutes ago      127 MB
<none>                   <none>              1572110032c9        21 minutes ago      127 MB
<none>                   <none>              404be961b34a        23 minutes ago      127 MB
<none>                   <none>              fd000d9e47bf        26 minutes ago      127 MB
<none>                   <none>              a3d647101bd1        7 hours ago         127 MB
<none>                   <none>              ca2fad92f95a        7 hours ago      

In [49]:
docker push kraeml/cowsay

The push refers to a repository [docker.io/kraeml/cowsay]



latest: digest: sha256:e4c71cd2185e66bff338104e7d9f29fb32daa87ea19ce00679b8a22324b1421f size: 948


##  Das offizielle Redis-Image verwenden

In [50]:
docker pull redis

Using default tag: latest
latest: Pulling from library/redis







Digest: sha256:38e873a0db859d0aa8ab6bae7bcb03c1bb65d2ad120346a09613084b49185912
Status: Downloaded newer image for redis:latest


In [51]:
docker run --name myredis -d redis

ebabe47e878cc4516873cea1490fda6814db82e3562a60293de102eafc43ad1c


Auf dem Terminal

```bash
docker run --rm -it --link myredis:redis redis /bin/bash
root@b5db049be09b:/data# redis-cli -h redis -p 6379
redis:6379> ping
PONG
redis:6379> set "abc" 123
OK
redis:6379> get "abc"
"123"
redis:6379> exit
root@b5db049be09b:/data#
```

Im Terminal

```bash
vagrant@docker:~/projects/cowsay$ docker run --rm -it --link myredis:redis redis /bin/bash
root@b925a1fbb9a5:/data# redis-cli -h redis -p 6379
redis:6379> set "persitence" "test"
OK
redis:6379> save 
OK
redis:6379> exit
root@b925a1fbb9a5:/data# exit
exit
```

In [52]:
docker run --rm --volumes-from myredis -v \
    $(pwd)/backup:/backup debian cp /data/dump.rdb /backup/

In [54]:
ls backup

dump.rdb


In [55]:
docker stop myredis

myredis


In [56]:
docker rm -v myredis

myredis


In [57]:
docker rm $(docker ps -aq)

f4f8e69fd467
4021677ba431
998911972d0f
da81384021fc
4c00f7a9f1f3
b813573f7a09
da5a70fda7f9
48d86be15e38


In [58]:
docker images

REPOSITORY               TAG                 IMAGE ID            CREATED             SIZE
kraeml/cowsay            latest              b4a1cd81ef44        18 minutes ago      171 MB
test/cowsay-dockerfile   latest              4a3a51ff0663        26 minutes ago      171 MB
<none>                   <none>              56452449687c        26 minutes ago      171 MB
test/cowsayimage         latest              a24a6ed41f5b        27 minutes ago      171 MB
<none>                   <none>              1535d5438a34        39 minutes ago      127 MB
<none>                   <none>              1572110032c9        40 minutes ago      127 MB
<none>                   <none>              404be961b34a        41 minutes ago      127 MB
<none>                   <none>              fd000d9e47bf        45 minutes ago      127 MB
<none>                   <none>              a3d647101bd1        7 hours ago         127 MB
<none>                   <none>              ca2fad92f95a        7 hours ago      